In [ ]:
import streamlit as st
import pandas as pd
import folium
from folium.plugins import HeatMap
from sqlalchemy import create_engine
import utm
from streamlit_folium import folium_static
import branca.colormap as cm

# Detalles de la conexión
server = '192.168.200.31'
database = 'jmineops'
username = 'tinformacion'
password = 'Timeinlondon$'
conn_str = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
engine = create_engine(conn_str)

# Consulta SQL para seleccionar todos los datos de la tabla
query = """
SELECT [time]
      ,[Northing] AS j
      ,[Easting] AS i
      ,[Elevation]
      ,[speed] AS Speed
      ,[full_name]
      ,[Alarma]
  FROM [jmineops].[dbo].[T_sensors_speed_coord]
"""

# Ejecutar la consulta y cargar los datos en un DataFrame
data = pd.read_sql(query, engine)

# Convertir las coordenadas UTM a latitud y longitud
def utm_to_latlon(row):
    lat, lon = utm.to_latlon(row['i'], row['j'], 13, 'N')  # Ajustar la zona y el hemisferio según sea necesario
    return pd.Series([lat, lon])

data[['latitude', 'longitude']] = data.apply(utm_to_latlon, axis=1)

# Filtrar datos para eliminar coordenadas inválidas
valid_lat_range = (18, 22)
valid_lon_range = (-107, -100)
data = data[(data['latitude'].between(*valid_lat_range)) & (data['longitude'].between(*valid_lon_range))]

# Convertir la columna 'time' a datetime
data['time'] = pd.to_datetime(data['time'])

# Eliminar filas con valores faltantes y velocidades mayores a 50 km/h
data = data.dropna(subset=['latitude', 'longitude', 'Speed', 'Alarma', 'Elevation'])
data = data[data['Speed'] <= 50]

# Título de la aplicación
st.title('Visualización de Mapas de Calor')

# Filtros de fecha y empleado
start_date = st.sidebar.date_input('Fecha de inicio', data['time'].min().date())
end_date = st.sidebar.date_input('Fecha de fin', data['time'].max().date())
employee = st.sidebar.selectbox('Empleado', ['Todos'] + list(data['full_name'].unique()))

# Filtrar datos según los filtros seleccionados
filtered_data = data[
    (data['time'] >= pd.to_datetime(start_date)) &
    (data['time'] <= pd.to_datetime(end_date))
]

if employee != 'Todos':
    filtered_data = filtered_data[filtered_data['full_name'] == employee]

# Mapa de velocidad
st.header('Mapa de Calor: Velocidad')
speed_map_center = [filtered_data['latitude'].mean(), filtered_data['longitude'].mean()]
speed_map = folium.Map(location=speed_map_center, zoom_start=13)
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Esri Satellite',
    overlay=False,
    control=True
).add_to(speed_map)

heatmap_data = filtered_data[['latitude', 'longitude', 'Speed']].dropna().values.tolist()
HeatMap(
    heatmap_data,
    min_opacity=0.5,
    radius=12,
    blur=10,
    max_zoom=1,
    gradient={0.0: 'blue', 0.5: 'lime', 1.0: 'red'}
).add_to(speed_map)

colormap = cm.LinearColormap(['blue', 'lime', 'red'], vmin=0, vmax=50)
colormap.caption = 'Velocidad (km/h)'
colormap.add_to(speed_map)

folium_static(speed_map)

# Mapa de alarmas
st.header('Mapa de Calor: Alarmas')
alarma = st.sidebar.selectbox('Alarma', ['Todas'] + list(data['Alarma'].unique()))
if alarma != 'Todas':
    filtered_data = filtered_data[filtered_data['Alarma'] == alarma]

alarm_map_center = [filtered_data['latitude'].mean(), filtered_data['longitude'].mean()]
alarm_map = folium.Map(location=alarm_map_center, zoom_start=13)
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Esri Satellite',
    overlay=False,
    control=True
).add_to(alarm_map)

heatmap_data = filtered_data[['latitude', 'longitude']].dropna().values.tolist()
HeatMap(
    heatmap_data,
    min_opacity=0.5,
    radius=12,
    blur=10,
    max_zoom=1,
    gradient={0.0: 'blue', 0.5: 'lime', 1.0: 'red'}
).add_to(alarm_map)

colormap = cm.LinearColormap(['blue', 'lime', 'red'], vmin=0, vmax=filtered_data['Alarma'].nunique())
colormap.caption = 'Alarmas'
colormap.add_to(alarm_map)

folium_static(alarm_map)

# Mapa de elevación
st.header('Mapa de Calor: Elevación')
elevation_map_center = [filtered_data['latitude'].mean(), filtered_data['longitude'].mean()]
elevation_map = folium.Map(location=elevation_map_center, zoom_start=13)
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Esri Satellite',
    overlay=False,
    control=True
).add_to(elevation_map)

heatmap_data = filtered_data[['latitude', 'longitude', 'Elevation']].dropna().values.tolist()
HeatMap(
    heatmap_data,
    min_opacity=0.5,
    radius=12,
    blur=10,
    max_zoom=1,
    gradient={0.0: 'blue', 0.5: 'lime', 1.0: 'red'}
).add_to(elevation_map)

colormap = cm.LinearColormap(['blue', 'lime', 'red'], vmin=filtered_data['Elevation'].min(), vmax=filtered_data['Elevation'].max())
colormap.caption = 'Elevación (m)'
colormap.add_to(elevation_map)

folium_static(elevation_map)
